In [1]:
import pandas as pd # analyze data
import string
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

C:\Users\aoral\anaconda3\envs\tensorflow-gpu\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\aoral\anaconda3\envs\tensorflow-gpu\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\aoral\anaconda3\envs\tensorflow-gpu\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
df = pd.read_csv('')
pd.set_option('display.max_rows', None) # display all rows in df

In [ ]:
df.head(100)

In [ ]:
df.tail(100)

In [5]:
def preprocess_name(name, max_length=20):
    # Convert non-string values (like NaN) to an empty string
    if not isinstance(name, str):
        name = ''

    # Define a dictionary mapping characters to integers
    char_to_int = {ch: i+1 for i, ch in enumerate(string.ascii_lowercase)}

    # Convert name to lowercase
    name = name.lower()

    # Convert each character in the name to its corresponding integer
    encoded = [char_to_int.get(ch, 0) for ch in name]

    # Pad or trim the encoded name to the fixed length
    encoded = encoded[:max_length] + [0] * (max_length - len(encoded))

    return np.array(encoded)


In [6]:
# Preprocess names (this is simplified; you might need more complex preprocessing)
#X = df['Name'].apply(preprocess_name)  # Convert names into a numerical format
X = np.array(df['Name'].apply(preprocess_name).tolist())
y = df['Country']

In [7]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [8]:
# Train a model
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [9]:
# Predict and evaluate the model
predictions = model.predict(X_test)
print(accuracy_score(y_test, predictions))

0.8799383853271104


In [11]:
from joblib import dump

# Assuming your model is named 'model'
dump(model, 'rfc_predict_nation_by_names.joblib')

['rfc_predict_nation_by_names.joblib']

In [ ]:
from joblib import load

model = load('rfc_predict_nation_by_names.joblib')

In [17]:
# Function to make a prediction
"""
def predict_nationality(model, name):
    # Preprocess the name
    processed_name = preprocess_name(name)
    
    # Reshape to match the input format of the model (1 sample, n features)
    processed_name = processed_name.reshape(1, -1)
    
    # Make a prediction
    predicted_nationality = model.predict(processed_name)
    
    return predicted_nationality[0]
"""

def predict_nationality(model, name, confidence_threshold=0.5):
    processed_name = preprocess_name(name)
    processed_name = processed_name.reshape(1, -1)

    # Get the model's probability estimates
    probabilities = model.predict_proba(processed_name)[0]

    # Get the most likely nationality and its probability
    max_probability = np.max(probabilities)
    if max_probability < confidence_threshold:
        return "Unknown"

    predicted_nationality = model.classes_[np.argmax(probabilities)]
    return predicted_nationality

In [20]:
# Test the model with a new name
test_name = "Alikhan"
predicted_nationality = predict_nationality(model, test_name)
print(f"The predicted nationality for {test_name} is: {predicted_nationality}")

The predicted nationality for Alikhan is: Azerbaijan
